In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper

driver = PySCFDriver(
    atom='H .0 .0 .0; H .0 .0 0.735',
    unit=DistanceUnit.ANGSTROM,
    basis='sto3g',
)
problem = driver.run()

# Get the fermionic Hamiltonian (Hatree-Fock approximation, STO-3G, fermionic creation and annihilation, hopping-occupation-interaction, etc.)
fermionic_hamiltonian = problem.hamiltonian.second_q_op()
print("FERMIONIC HAMILTONIAN: \n")
print(fermionic_hamiltonian)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian (Parity mapping, could be Jordan-Wigner or Bravyi-Kitaev)
mapper = ParityMapper(num_particles=problem.num_particles)
qubit_hamiltonian = mapper.map(problem.hamiltonian.second_q_op())
print("\n\nQUBIT HAMILTONIAN: \n")
print(qubit_hamiltonian)

FERMIONIC HAMILTONIAN: 

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.3322908651276482 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.3322908651276482 * ( +_0 +_3 -_3 -_0 )
+ 0.09046559989211572 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211572 * ( +_0 +_1 -_0 -_1 )
+ 0.09046559989211572 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211572 * ( +_0 +_3 -_2 -_1 )
+ 0.09046559989211572 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211572 * ( +_1 +_1 -_0 -_0 )
+ 0.09046559989211572 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211572 * ( +_1 +_3 -_2 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.34928686136600884 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.34928686136600884 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.3322908651276482 * ( +_2 +

In [ ]:
from lib.hamiltonian_builder import make_hydrogen_hamiltonian_hea
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import L_BFGS_B

# SIMULATOR
optimizer = L_BFGS_B()
estimator = Estimator()
distance_shots=[0.1, 0.2, 0.5, 0.735, 1.0, 1.5, 2.0, 3.0, 4.0]
convergences2 = []
# iterations_of_convergences = [[] for _ in range(len(distance_shots))]
nits2=[]

for i in distance_shots:
    (qubit_hamiltonian, ansatz) = make_hydrogen_hamiltonian_hea(i)
    def estimate_energy(parameters):
        job = estimator.run(
            [(ansatz, qubit_hamiltonian, parameters)]
        )
        result = job.result()[0].data.evs
        # iterations_of_convergences[i].append(result)
        return result

    initial_point = np.zeros(ansatz.num_parameters)
    result = optimizer.minimize(fun=estimate_energy, x0=initial_point)
    nits2.append(result.nit)
    convergences2.append(result)

energies = [res.fun for res in convergences2]
plt.plot(distance_shots, energies, marker='o', linestyle='-')
plt.xlabel('Distances (Å)')
plt.ylabel('Energy (Hartree)')
plt.title('Ground state energies of $H_2$')
plt.show()

TypeError: BaseEstimator.run() missing 1 required positional argument: 'observables'